# Tree Classifiers

In [1]:
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

plt.style.use("Solarize_Light2")

In [2]:
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST Original')
mnist

/Users/rajatrasal/anaconda3/envs/crypto_time_series/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/Users/rajatrasal/anaconda3/envs/crypto_time_series/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


{'DESCR': 'mldata.org dataset: mnist-original',
 'COL_NAMES': ['label', 'data'],
 'target': array([0., 0., 0., ..., 9., 9., 9.]),
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

In [3]:
from sklearn.model_selection import train_test_split

X, y = mnist['data'], mnist['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"""X_train: {X_train.shape} X_test: {X_test.shape},
y_train: {y_train.shape} y_test: {y_test.shape}""")

X_train: (52500, 784) X_test: (17500, 784),
y_train: (52500,) y_test: (17500,)


In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print(f"""X_train: {X_train.shape} X_val: {X_val.shape},
y_train: {y_train.shape} y_val: {y_val.shape}""")

X_train: (47250, 784) X_val: (5250, 784),
y_train: (47250,) y_val: (5250,)


## Decision Tree Classifier

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

A decision tree is a white box model in that it is fairly easy to interpret the decisions being made**[x]**. 

Each node in the decision tree has a number of attributes:
* **Samples** - then number of training instances applicable at that node.
* **Value** - the number of training instances of each class this node applies to, e.g. [0, 10, 20], if there are 3 classes we are classifying by.
* **Impurity** - a measure of if all training instances this node apply to fall under the same class. This is done by either calculating the gini or entropy of the node. This is the value which is being minimised when optimising the decision tree.

![IMAGE OF DECISION TREE FROM GRAPHVIZ]()

**Predictions** are made by following the decision tree down to the leaves for a particular training instance. At the leaf, we look at the *value* attribute. The prediction is then made as follows:

```
# Find the probabilities of each class being the correct.
percent_proba = map(enumerate(value), lambda i, x: (i, x/sum(value)))
# Select the class with the highest probability.
class, percent = max(percent_proba, key=lambda x: x[1])
```

**Impurity** is determined by 2 main methods:
* *Entropy* - based on Shannon's Information Theory. We use this is a measure of the average information content of a message, hence entropy is 0 when a node only applies to values of 1 class. $$H_i = - \sum_{k=1}^n \space p_{i, k} log(p_{i, k})$$ where $p_{i, k} =$ ```nodes[i].values[k] / sum(nodes[i].values)```.
* *Gini* - This purely looks at the class probabilities. $$G_i = 1 - \sum_{k=1}^n \space (p_{i, k})^2$$ where $p_{i, k}$ is defined in the same way as for entropy.

In most cases, it doesn't matter which model loss function we pick. Gini is slightly faster but in some cases entropy can result in more balanced trees.

**CART**
Classification and Regression Tree Algorithm (CART) is the training algorithm used by scikit-learn. At each node, the algorithm first splits the dataset into 2 sets by choosing a single feature $k$ and a threshold $t_k$, based on which $(k, t_k)$ pair minimise the impurity the most.

$$J(k, t_k) = \frac{m_{left}}{m}I_{left} + \frac{m_{right}}{m}I_{right}$$

This continued until we reach a specified max depth.

---
[x] This is in contract to neural network where there is no clear or intuitive way to decide why a feature training decision was made.

### PCA
Unlike SVMs, we want to reduce the feature space of the decision tree, or else the CART algorithm will overfit the training set**[]**. We can do this using PCA.

From Tensorflow and Sklearn book, to preserve about 95% of MNIST's variance, we need to reduce its feature space down from 784 to 150.

---
[] With an SVM we often want to increase the feature space, by using an RBF kernel for example, so as to the increase the chances of being able to fit a linear model which accurately fits the training set. PCA may still be useful as a compression method to speed up the overall computations of an RBF kernel or linear SVM.

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=154)
# Do the image comparison to show how the compressed image is different

In [17]:
dt_clf = Pipeline([
    ("pca", PCA(n_components=154)),
    ("clf", DecisionTreeClassifier(random_state=42))
])

params = {"clf__max_depth": 20}

dt_clf_1 = dt_clf.set_params(**params)
dt_clf_1.fit(X_train, y_train)

y_train_pred = cross_val_predict(dt_clf_1, X_train, y_train, cv=5)
y_val_pred = cross_val_predict(dt_clf_1, X_val, y_val, cv=5)
print(f"train accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"val accuracy: {accuracy_score(y_val, y_val_pred)}")

train accuracy: 0.8220740740740741
val accuracy: 0.708952380952381


## Random Forest Classifier

## Extra Trees Classifier

## References

[] Gini vs Entropy loss functions - https://sebastianraschka.com/faq/docs/decision-tree-binary.html